<a href="https://colab.research.google.com/github/StehCastro/Projeto-Analise-de-credito/blob/main/An%C3%A1lise_de_Cr%C3%A9dito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Definição do Problema
Uma empresa que oferece empréstimo a pessoas físicas, necessita saber, com base nos dados de seus clientes, quais novos clientes poderão ou não ter acesso ao crédito solicitado (empréstimo).

Pensando no problema da empresa, criei um modelo utilizando *Machine Learning*, com base nas informações históricas dos clientes dessa empresa (cliente que pediram empréstimo que pagaram e que não pagaram), onde o objetivo é com base nesse histórico, prevermos se possíveis novos clientes irão ou não arcar com seus compromissos.
O produto final será uma aplicação web, onde a entrada são os dados do possível novo cliente e com base nesses dados, o modelo irá informar se pode ou não liberar o crédito solicitado.









## 2. Preparação dos Dados


In [2]:
import os
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Carregando a Fonte de Dados (Análise de Crédito)
#
*  *.csv (“comma-separated-values” - valores separados por vírgulas)

In [3]:
data = pd.read_csv('loan.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'loan.csv'

Disposição dos dados, 5 primeiras linhas.


In [ ]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Quantidade de Linhas e Colunas, Tipos das Colunas, Tamanho entre outras informações.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


Verificando o balanceamento dos Labels

In [ ]:
data.Loan_Status.value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

In [ ]:
data2 = data[data.Loan_Status=='Y'].sample(200)

Abaixo, anexa as 200 amostras da **classe Y** com os registros da **classe N**, em um dataframe que chamamos de **data**

In [ ]:
data = data2.append(data[data.Loan_Status=='N'].sample(192))

Verificando novamente o equilíbrio entre as classes.

In [ ]:
data.Loan_Status.value_counts()

Y    200
N    192
Name: Loan_Status, dtype: int64

### Checando Missing Values (Valores nulos)

In [ ]:
data.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [ ]:
# Exemplo - Contagem de tipos de Generos
data.Gender.value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [ ]:
data['Gender'] = data['Gender'].fillna('Male')

In [ ]:
data['Married'] = data['Married'].fillna('No')

In [ ]:
data['Dependents'] = data['Dependents'].fillna('0')

In [ ]:
data['Self_Employed'] = data['Self_Employed'].fillna('No')

In [ ]:
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].mean())

In [ ]:
# Exemplo - Contagem de tipos de Generos
data.Credit_History.value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [ ]:
data['Credit_History'] = data['Credit_History'].fillna(1.0)

In [ ]:
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mean())

#### Checando novamente Missing Values


In [ ]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

### Transformando dados categóricos

Realizando os mapeamentos e substituindo os valores das colunas no dataframe (data)

In [ ]:
gender_values = {'Female' : 0, 'Male' : 1}
married_values = {'No' : 0, 'Yes' : 1}
education_values = {'Graduate' : 0, 'Not Graduate' : 1}
employed_values = {'No' : 0, 'Yes' : 1}
dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
loan_values = {'Y':1,'N':0}
data.replace({'Gender': gender_values,
                 'Married': married_values,
                 'Education': education_values,
                 'Self_Employed': employed_values,
                 'Dependents': dependent_values,
                 'Loan_Status': loan_values
                }, inplace=True)

Executando o comando drop para realizar essa exclusão

In [ ]:
data.drop(['Loan_ID','CoapplicantIncome','Loan_Amount_Term','Credit_History','Property_Area'],axis=1,inplace=True)

Vamos verificar como ficou o dataframe após a nossa seleção de atributos (*features*)

In [ ]:
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Status
0,1,0,0,0,0,5849,146.412162,1
1,1,1,1,0,0,4583,128.000000,0
2,1,1,0,0,1,3000,66.000000,1
3,1,1,0,1,0,2583,120.000000,1
4,1,0,0,0,0,6000,141.000000,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

Instanciando o classificador *Random Forest*

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=100,min_samples_split=2)

Dividindo o conjunto de dados (Valores - X (variáveis independentes) e Rótulos - y (*target* ou variávies dependentes))

In [ ]:
X = data.drop('Loan_Status',axis=1)
y = data['Loan_Status']

dividindo o conjunto em Treinamento e Teste, utilizando o *train_test_split*

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y,test_size=0.20,random_state=42)

In [ ]:
clf_rf.fit(X_treino,y_treino)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Importando biblioteca de métricas do *sklearn*

In [ ]:
from sklearn import metrics

In [ ]:
print (pd.crosstab(y_teste, clf_rf.predict(X_teste), rownames=['Real'], colnames=['Predito'], margins=True), '')

(Predito   0    1  All
Real                 
0         6   37   43
1         8   72   80
All      14  109  123, '')


 **Relatório de Classificação**

In [ ]:
print (metrics.classification_report(y_teste,clf_rf.predict(X_teste)))

              precision    recall  f1-score   support

           0       0.43      0.14      0.21        43
           1       0.66      0.90      0.76        80

   micro avg       0.63      0.63      0.63       123
   macro avg       0.54      0.52      0.49       123
weighted avg       0.58      0.63      0.57       123



In [ ]:
from sklearn.externals import joblib

Persistindo o modelo em disco com o comando *dump* do **joblib**.

In [ ]:
joblib.dump(clf_rf, 'model/model.pkl')

['/content/drive/My Drive/0. Business/2. Consultoria em Dados/2. IA, ML/0. Scripts, Exemplos, Cursos/Scripts exemplos/Deploy em Produc\xcc\xa7a\xcc\x83o/model.pkl']

#### Carregando o modelo a partir do disco para a memória no servidor Web

In [ ]:
model = joblib.load('model.pkl')

verificando os atributos do modelo...

In [ ]:
print("Atributos do Modelo:\n\nClasses:{}\n\nEstimators:{}\n\nParametros:{}".format(model.classes_,model.n_estimators,model.base_estimator))

Atributos do Modelo:

Classes:[0 1]

Estimators:100

Parametros:DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


### Teste de Classificação

Na página devemos informar algumas informações dos possíveis clientes, como:
- Sexo (Masculino(1) e Feminino(0))
- Número de Dependentes
- Casado (Sim(1) ou Não(0))
- Grau de Instrução (Graduado(1) ou Não Graduado(0))
- Trabalha por conta própria (Sim(1) ou Não(2))
- Rendimento (rendimentos do cliente)
- Valor do Emprestimo (valor que o cliente quer emprestado)

A ordem que vai para o modelo é: Gender(Sexo), Married (Casado), Dependents (Número de Dependentes), Education (Grau de Instrução), Self_Employed (Trabalha por conta própria), ApplicantIncome (Rendimentos) e	LoanAmount (Valor do Emprestimo)

Realizando um testes com as seguintes informações:
- Gender(Sexo) = 1
- Married (Casado) = 1
- Dependents (Número de Dependentes) = 3
- Education (Grau de Instrução) = 0
- Self_Employed (Trabalha por conta própria) = 0
- ApplicantIncome (Rendimentos) = 9504
- LoanAmount (Valor do Emprestimo) = 275

In [ ]:
teste = np.array([[1,1,3,0,0,9504,275.0]])

Rondando o Modelo pra prever o teste

In [ ]:
model.predict(teste)

array([0])

O Modelo nos retornou as seguintes probabilidades:

In [ ]:
model.predict_proba(teste)

array([[0.56, 0.44]])

Ou seja, o modelo informou com base nos dados de treinamento, que o cliente (do teste) tem a probabilidade de 56% de pagar o empréstimo, então ele irá liberar o crédito solicitado para esse cliente.

### Publicando na Web
Estrutura do Diretório
- Model
 - modelo gerado acima deve ficar nessa pasta (model.pkl)
- static
 - arquivos auxiliares da página
- templates
 - página principal (template.html)
- *srv.py*
 - Renderizando a página usando Flask

#### Publicando...

```
C:\Python>python srv.py

 * Serving Flask app "srv" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://localhost:5500/ (Press CTRL+C to quit)

```

##### Página Publicada
![alt text](https://blogdozouza.files.wordpress.com/2020/02/producao.png)



